In [2]:
# Bibliotecas
from bibkmis.auxkmis import *
from bibkmis.typeskmis import SOLUCAO, KMIS

import ast                      # Ler os litearias de tipos simples
import os                       # Controle de pastas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np              # Principal para ferramentas matematicas
from tqdm import tqdm           # Barrinha de progresso

In [5]:
# Restaurando Instâncias para consulta
dictI = {
  "id": [], "kmis": [], "p": [], "k": [],
  "|L|": [], "|R|": [], "L": [], 'temSol': [], 'classe': []
}
dfI = pd.DataFrame(columns = list(dictI.keys()))
# ==== Carregamento de Instâncias Salvas =====
conv = {
  'L'     : ast.literal_eval,
  'temSol': ast.literal_eval,
  'L_b14' : ast.literal_eval,
  'Llabel': ast.literal_eval,
  'Rlabel': ast.literal_eval
}
try:
  dfI = pd.read_csv('instancias.csv', converters=conv)
  print(f'Leitura de instancias.csv ({dfI.shape[0]} linhas) bem sucedida.')
except:
  print('\n\n\t\tArquivo instancias não encontrado!!\n\n')
  assert dfI.shape[0]>0 , "Sem instâncias não continua! Peque o arquivo 'instancias.csv'."

# Reinstanciar objetos KMIS a partir das linhas do CSV
dictI['kmis_b14'] = []
with tqdm(total = dfI.shape[0], smoothing = 0.001, desc="Reconstruindo Instâncias") as pbar:
  for _, row in dfI.iterrows():
    kmis = KMIS(row['|L|'], row['|R|'], row['p'], row['k'], row['L'])
    kmis_reduzido = KMIS(row['|L|_b14'], row['|R|_b14'], row['p'], row['k'], row['L_b14'])
    kmis_reduzido.Llabel = row['Llabel_b14']
    kmis_reduzido.Rlabel = row['Rlabel_b14']
    dictI['kmis'].append(kmis)
    dictI['kmis_b14'].append(kmis_reduzido)
    pbar.update(1)

dfI['kmis'] = dictI['kmis']
dfI['kmis_b14'] = dictI['kmis_b14']

Leitura de instancias.csv (540 linhas) bem sucedida.


Reconstruindo Instâncias: 100%|██████████| 540/540 [00:02<00:00, 211.59it/s] 


In [ ]:
# Leitura das tabelas:
try:
  dfAT = pd.read_csv('teste_parametros.csv')
  dfRT = pd.read_csv('resultados.csv')
  dfIRT = pd.read_csv('resultados_reduzidas.csv')
except:
  print('Tem arquivo faltante!')

In [ ]:
""" Analise dos resultados geral"""

# ===========================================================================================
assert isinstance(dfRT, pd.DataFrame), '\t ⚠️ DataFrame do teste final não definido.'
dfR_hi = dfRT.groupby(['idH', 'idI'])[['val', 'time']].apply(junta_repeticoes).reset_index()
dfR_h  = dfR_hi.groupby(['idH'])[['vmin', 'vavg', 'vmax', 'tavg']].apply(medias).reset_index()
dfR_i  = dfR_hi.groupby(['idI'])[['vmin', 'vavg', 'vmax', 'tavg']].apply(melhor_por_instancia).reset_index()
# Comparações
cmp = dfR_hi.merge(dfR_i, on=['idI'])
cmp['eq_vmin'] = cmp['vmin'] == cmp['vmin_max']
cmp['eq_vmax'] = cmp['vmax'] == cmp['vmax_max']
cmp['eq_vavg'] = np.isclose(cmp['vavg'], cmp['vavg_max'], atol=1e-4)
cmp['eq_tavg'] = np.isclose(cmp['tavg'], cmp['tavg_min'], atol=1e-4)

# Contagem e merge final
cnt = cmp.groupby(['idH'])[['eq_vmin', 'eq_vavg', 'eq_vmax', 'eq_tavg']].sum().rename(columns=lambda c: 'cnt_' + c[3:]).reset_index()
dfR = dfR_h.merge(cnt, on=['idH'])
df_limites = limites_argumento(dfR).to_frame().T
df_score = dfR.merge(df_limites, how='cross')
dfR['score'] = df_score.apply(score_time_off, axis=1)

dfR.to_csv('resultados_avaliados_geral.csv', index = False)
print(dfR)

In [ ]:
""" Analise dos resultados por classe"""
# ===========================================================================================
dfRT_c = dfRT.merge(dfI[['id', 'classe']], left_on='idI', right_on='id').drop('id', axis=1)
dfR_hi = dfRT_c.groupby(['classe', 'idH', 'idI'])[['val', 'time']].apply(junta_repeticoes).reset_index()
dfR_i  = dfR_hi.groupby(['idI'])[['vmin', 'vavg', 'vmax', 'tavg']].apply(melhor_por_instancia).reset_index()
dfR_h_c  = dfR_hi.groupby(['classe','idH'])[['vmin', 'vavg', 'vmax', 'tavg']].apply(medias).reset_index()
cmp = dfR_hi.merge(dfR_i, on=['idI'])
cmp['eq_vmin'] = cmp['vmin'] == cmp['vmin_max']
cmp['eq_vmax'] = cmp['vmax'] == cmp['vmax_max']
cmp['eq_vavg'] = np.isclose(cmp['vavg'], cmp['vavg_max'], atol=1e-4)
cmp['eq_tavg'] = np.isclose(cmp['tavg'], cmp['tavg_min'], atol=1e-4)
cnt = cmp.groupby(['classe', 'idH'])[['eq_vmin', 'eq_vavg', 'eq_vmax', 'eq_tavg']].sum().rename(columns=lambda c: 'cnt_' + c[3:]).reset_index()
dfR_c = dfR_h_c.merge(cnt, on=['classe', 'idH'])
df_limites = dfR_c.groupby(['classe'])[dfR_c.columns[2:]].apply(limites_argumento)
df_score = dfR_c.merge(df_limites, left_on='classe', right_index=True)
dfR_c['score'] = df_score.apply(score_time_off, axis=1)

Score_PerClass = pd.DataFrame({'idH':dfR_c['idH'].unique()})
for i in dfR_c['classe'].unique():
  subset = pd.DataFrame(dfR_c[dfR_c['classe'] == i][['classe', 'idH', 'score']])
  subset = subset.rename(columns = {'score':f'score_{i}'})
  # display(subset)
  Score_PerClass = Score_PerClass.merge(subset, on='idH', how='left').drop('classe',axis=1)
# Score_PerClass = Score_PerClass.drop(['score_C2','score_C6'], axis=1)
Score_PerClass['avg'] = Score_PerClass.apply(lambda x: x[1:].mean(), axis=1)
Score_PerClass.to_csv('resultados_avaliados_por_classe.csv', index = False)
print(Score_PerClass.sort_values('avg', ascending=False))